In [2]:
!pip install keras

    100% |████████████████████████████████| 307kB 2.3MB/s ta 0:00:01


In [16]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential

from keras.layers import Dense, Dropout
# from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [17]:
df_train = pd.read_csv('./notebooks/data/asilomar_nov182017_all.csv')
# this is actually [u1, v1, u2, v2]
X = df_train[['u','v','w','h']]
y = df_train['distance']

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.30)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [19]:
NUM_FEATURES = len(X.columns)

In [20]:
def dist_regression_model():
    model = Sequential()
    model.add(Dense(NUM_FEATURES*4, input_dim=NUM_FEATURES, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
#     model.add(BatchNormalization())
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
    model.add(Dense(NUM_FEATURES*4, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [21]:
model = dist_regression_model()

In [32]:
model.fit(x=X_train, 
          y=y_train, 
          batch_size=32, 
          epochs=30, validation_data = (X_val, y_val))

Train on 11088 samples, validate on 4753 samples
Epoch 1/30
11088/11088 [==============================] - 1s 47us/step - loss: 86.1270 - val_loss: 75.6218
Epoch 2/30
11088/11088 [==============================] - 0s 42us/step - loss: 85.7471 - val_loss: 87.2043
Epoch 3/30
11088/11088 [==============================] - 0s 41us/step - loss: 86.3141 - val_loss: 86.7846
Epoch 4/30
11088/11088 [==============================] - 0s 44us/step - loss: 86.9812 - val_loss: 76.4634
Epoch 5/30
11088/11088 [==============================] - 0s 42us/step - loss: 85.0524 - val_loss: 75.6891
Epoch 6/30
11088/11088 [==============================] - 0s 42us/step - loss: 85.0799 - val_loss: 78.1240
Epoch 7/30
11088/11088 [==============================] - 0s 42us/step - loss: 84.5779 - val_loss: 75.0212
Epoch 8/30
11088/11088 [==============================] - 0s 42us/step - loss: 86.4133 - val_loss: 74.8697
Epoch 9/30
11088/11088 [==============================] - 1s 50us/step - loss: 84.5353 - val_lo

In [35]:
def evaluate_model(model, X_val, y_val):
    # training error
    pred = model.predict(X_val)

    error = y_val[:,np.newaxis]-pred
    print 'MSE: {}'.format(np.mean(np.square(error)))

    # mean absolute percentage
    mean_abs_percentage = np.mean(np.abs(error)/y_val[:,np.newaxis])
    print 'Mean Abs Percentage:{}'.format(mean_abs_percentage)

evaluate_model(model, X_val, y_val)

MSE: 68.9576788146
Mean Abs Percentage:0.0981642733332


In [11]:
# test set
X_val_np = X_val
y_val_np = y_val
pred = model.predict(X_val_np)

# mean square error
error = y_val_np[:,np.newaxis]-pred
print np.mean(np.square(error))

# mean absolute percentage
mean_abs_percentage = np.mean(np.abs(error)/y_val_np[:,np.newaxis])
print mean_abs_percentage

99.57645189847871
0.11868603329395497


In [12]:
print error[:10]

[[ 2.56878949]
 [ 3.12520735]
 [ 1.22812887]
 [ 9.53163675]
 [-1.83286173]
 [14.36817481]
 [ 1.76770871]
 [ 0.75228073]
 [-0.21887892]
 [ 2.00120417]]


In [13]:
bbox = np.array([[147.,  295.,147+331.,295.+173.]],np.float32)
bbox = scaler.transform(bbox)
print bbox.shape
print model.predict(bbox)

(1, 4)
[[13.313145]]


In [14]:
scaler.inverse_transform(X_train[100,:])


array([107., 255., 402., 475.])

In [15]:
bbox = np.array([[240,300,46,39]],np.float32)
scaler.transform(bbox)

array([[-0.1473077,  4.6086993, -4.7980194, -5.175077 ]], dtype=float32)